# Import Packages

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Normalize
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import os
import pandas as pd
from PIL import Image
import numpy as np

from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

# Loading Data

In [2]:
# Define the training and testing folder paths
train_path = [
    'archive/training/glioma_tumor',
    'archive/training/meningioma_tumor',
    'archive/training/no_tumor',
    'archive/training/pituitary_tumor'
]

test_path = [
    'archive/testing/glioma_tumor',
    'archive/testing/meningioma_tumor',
    'archive/testing/no_tumor',
    'archive/testing/pituitary_tumor'
]

# Initialize lists to hold image data and labels for training and testing
train_image_data = []
train_labels = []

test_image_data = []
test_labels = []

# Define a target image size for resizing (e.g., 224x224 pixels)
image_size = (224, 224)

# Function to process images and assign labels
def process_images(folders, label_set, image_data_list, label_list):
    for folder in folders:
        # Extract the label from the folder name (e.g., 'glioma_tumor')
        label = folder.split('/')[-1]  # Folder name as label (e.g., 'glioma_tumor')
        # Get the list of image files in the folder
        for filename in os.listdir(folder):
            if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
                # Open the image
                image_path = os.path.join(folder, filename)
                img = Image.open(image_path)
                
                # Resize the image to the target size
                img = img.resize(image_size)
                
                # Convert image to an array (now in the form of a 3D array)
                img_array = np.array(img)
                
                # Flatten the image array to turn it into a vector
                img_vector = img_array.flatten()
                
                # Append image data and its label
                image_data_list.append(img_vector)
                label_list.append(label)

# Process the training data
process_images(train_path, 'training', train_image_data, train_labels)

# Process the testing data
process_images(test_path, 'testing', test_image_data, test_labels)

# Create dataframes for training and testing datasets
train_df = pd.DataFrame(train_image_data)
train_df['label'] = train_labels

test_df = pd.DataFrame(test_image_data)
test_df['label'] = test_labels

# Display the first few rows of the training and testing datasets
print("Training Dataset:")
print(train_df.head())

print("\nTesting Dataset:")
print(test_df.head())

Training Dataset:
   0  1  2  3  4  5  6  7  8  9  ...  150519  150520  150521  150522  150523  \
0  0  0  0  0  0  0  0  0  0  0  ...       0       0       0       0       0   
1  0  0  0  0  0  0  0  0  0  0  ...       0       0       0       0       0   
2  0  0  0  0  0  0  0  0  0  0  ...       0       0       0       0       0   
3  0  0  0  0  0  0  0  0  0  0  ...       1       1       1       1       1   
4  0  0  0  0  0  0  0  0  0  0  ...       0       0       0       0       0   

   150524  150525  150526  150527         label  
0       0       0       0       0  glioma_tumor  
1       0       0       0       0  glioma_tumor  
2       0       0       0       0  glioma_tumor  
3       1       1       1       1  glioma_tumor  
4       0       0       0       0  glioma_tumor  

[5 rows x 150529 columns]

Testing Dataset:
   0  1  2  3  4  5  6  7  8  9  ...  150519  150520  150521  150522  150523  \
0  0  0  0  0  0  0  0  0  0  0  ...       0       0       0       0       0

In [42]:
target_column = 'label'

# Check class distribution
class_distribution = train_df[target_column].value_counts()

# Print the class distribution
print("Train Data Class Distribution:")
print(class_distribution)

# Calculate the percentage of each class
class_percentage = train_df[target_column].value_counts(normalize=True) * 100

# Print the class percentage
print("Train Data \nClass Percentage:")
print(class_percentage)

# Check class distribution
class_distribution = test_df[target_column].value_counts()

# Print the class distribution
print("Test Data Class Distribution:")
print(class_distribution)

# Calculate the percentage of each class
class_percentage = test_df[target_column].value_counts(normalize=True) * 100

# Print the class percentage
print("Test Data \nClass Percentage:")
print(class_percentage)

Train Data Class Distribution:
label
3    827
0    826
1    822
2    395
Name: count, dtype: int64
Train Data 
Class Percentage:
label
3    28.815331
0    28.780488
1    28.641115
2    13.763066
Name: proportion, dtype: float64
Test Data Class Distribution:
label
1    115
2    105
0    100
3     74
Name: count, dtype: int64
Test Data 
Class Percentage:
label
1    29.187817
2    26.649746
0    25.380711
3    18.781726
Name: proportion, dtype: float64


# VGG16 LoRA Model

In [23]:
class VGG16_LoRA(nn.Module):
    def __init__(self, num_classes=2, rank=8):
        super(VGG16_LoRA, self).__init__()

        # Define the VGG16 convolutional layers
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        )

        # Fully connected layers (adapted with LoRA)
        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

        # Apply LoRA (low-rank adaptation) to fully connected layers
        self.lora_fc1 = self.low_rank_adaptation(self.fc1, rank)
        self.lora_fc2 = self.low_rank_adaptation(self.fc2, rank)

    def forward(self, x):
        # Forward pass through the convolutional layers
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor for fully connected layers

        # Forward pass through the fully connected layers with LoRA applied
        x = self.lora_fc1(x)
        x = F.relu(x)
        x = self.lora_fc2(x)
        x = F.relu(x)
        x = self.fc3(x)

        return x

    def low_rank_adaptation(self, layer, rank):
        """
        Apply LoRA to a fully connected layer.
        Decompose the weight matrix into two low-rank matrices.
        """
        in_features = layer.in_features
        out_features = layer.out_features

        # Decompose the original weight matrix into low-rank matrices
        U = nn.Parameter(torch.randn(in_features, rank) * 0.01)  # Small initialization
        V = nn.Parameter(torch.randn(rank, out_features) * 0.01)  # Small initialization
        
        # LoRA layer does not directly modify the weights; instead, it modifies the forward pass
        # to use low-rank approximations during the forward pass.
        self.register_parameter(f"lora_U_{id(layer)}", U)
        self.register_parameter(f"lora_V_{id(layer)}", V)

        return layer

    def forward_lora(self, x, layer_name):
        """
        Apply LoRA-modified forward pass.
        """
        U = getattr(self, f"lora_U_{layer_name}")
        V = getattr(self, f"lora_V_{layer_name}")
        
        # Perform the matrix multiplication for LoRA
        return F.linear(x, U @ V.t())

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# Data Preprocessing

In [25]:
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

# Split features and labels
X_train = train_df.iloc[:, :-1].values  # All columns except 'label'
y_train = train_df['label'].values

X_test = test_df.iloc[:, :-1].values
y_test = test_df['label'].values

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Normalize the data to [0, 1] range (optional, depends on the model)
X_train_tensor /= 255.0
X_test_tensor /= 255.0

# Reshape the data from [batch_size, height * width * channels] to [batch_size, channels, height, width]
X_train_tensor = X_train_tensor.view(-1, 3, 224, 224)
X_test_tensor = X_test_tensor.view(-1, 3, 224, 224)

# Create a custom Dataset
class TensorDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Baseline Model

## Training

In [26]:
# Initialize the model
model = VGG16_LoRA(num_classes=4, rank=8).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop runtime 186 minutes
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# Call the training function
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 1.3765
Epoch [2/10], Loss: 1.3538
Epoch [3/10], Loss: 1.3506
Epoch [4/10], Loss: 1.3495
Epoch [5/10], Loss: 1.3476
Epoch [6/10], Loss: 1.3512
Epoch [7/10], Loss: 1.3503
Epoch [8/10], Loss: 1.3503
Epoch [9/10], Loss: 1.3493
Epoch [10/10], Loss: 1.3498


## Validation

In [29]:
# Evaluation function with torchmetrics
def evaluate_model(model, test_loader, device):
    model.eval()  # Set model to evaluation mode

    # Initialize metrics for multi-class classification
    accuracy = Accuracy(num_classes=4, task="multiclass").to(device)
    recall = Recall(num_classes=4, task="multiclass").to(device)
    precision = Precision(num_classes=4, task="multiclass").to(device)
    auroc = AUROC(num_classes=4, task="multiclass").to(device)

    # Track predictions and true labels
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Get predictions from logits (outputs)
            _, predicted = torch.max(outputs, 1)
            
            # Update the metrics
            accuracy.update(predicted, labels)
            recall.update(predicted, labels)
            precision.update(predicted, labels)
            auroc.update(outputs, labels)  # Use raw outputs for AUROC

            # Collect all labels and predictions for later metrics computation
            all_labels.append(labels.cpu())
            all_preds.append(predicted.cpu())

    # Compute all metrics
    accuracy_result = accuracy.compute()
    recall_result = recall.compute()
    precision_result = precision.compute()
    auroc_result = auroc.compute()

    # Reset metrics after computation
    accuracy.reset()
    recall.reset()
    precision.reset()
    auroc.reset()

    print(f"Accuracy: {accuracy_result:.2f}")
    print(f"Recall: {recall_result:.2f}")
    print(f"Precision: {precision_result:.2f}")
    print(f"AUROC: {auroc_result:.2f}")

    # Optionally, return all predictions if needed for further analysis
    return all_labels, all_preds

# Evaluate the model
all_labels, all_preds = evaluate_model(model, test_loader, device)

Accuracy: 0.25
Recall: 0.25
Precision: 0.25
AUROC: 0.49


# Hyperparameter Tuning

In [35]:
# Define hyperparameter values
learning_rates = [0.001, 0.005, 0.01]
batch_sizes = [32, 64]
# optimizers = ["adam", "adamw", "sgd"]
# epoch_sizes = [2, 3]

# Function to train and evaluate the model with given hyperparameters
def train_and_evaluate(params, model, train_dataset, test_loader):
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']
    # optimizer_choice = params['optimizer']
    num_epochs = 3 # params['epochs']

    # Update DataLoader with the new batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # if optimizer_choice == "adam":
    #     optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # elif optimizer_choice == "adamw":
    #     optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    # elif optimizer_choice == "sgd":
    #     optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    # Metrics
    precision_metric = Precision(task="multiclass", num_classes=4).to(device)
    recall_metric = Recall(task="multiclass", num_classes=4).to(device)
    # auroc_metric = AUROC(task="multiclass", num_classes=4).to(device)
    
    # Train the model
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")
    
    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_predictions.append(predicted)
            all_labels.append(labels)
    
    # Concatenate all predictions and labels for metrics
    all_predictions = torch.cat(all_predictions, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    # Calculate metrics
    accuracy = 100 * correct / total
    precision = precision_metric(all_predictions, all_labels).item()
    recall = recall_metric(all_predictions, all_labels).item()
    
    print(f"Accuracy: {accuracy:.2f}%, Precision: {precision:.4f}, Recall: {recall:.4f}")
    return accuracy, precision, recall

# Run manual grid search
best_accuracy = 0
best_params = None

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        # for optimizer_choice in optimizers:
            # for num_epochs in epoch_sizes:
                param_dict = {
                    "learning_rate": learning_rate,
                    "batch_size": batch_size,
                    # "optimizer": optimizer_choice,
                    # "epochs": num_epochs
                }
                print(f"Testing parameters: {param_dict}")
                
                # Initialize a fresh model for each combination
                model = VGG16_LoRA(num_classes=4, rank=8).to(device)
                
                # Evaluate current parameter combination
                accuracy, precision, recall = train_and_evaluate(param_dict, model, train_dataset, test_loader)
                print(f"Metrics - Accuracy: {accuracy:.2f}%, Precision: {precision:.4f}, Recall: {recall:.4f} for {param_dict}")
                
                # Track the best configuration based on accuracy
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = param_dict

print(f"Best Hyperparameters: {best_params}, Best Accuracy: {best_accuracy:.2f}%")

Testing parameters: {'learning_rate': 0.001, 'batch_size': 32}
Epoch [1/3], Loss: 1.4048
Epoch [2/3], Loss: 1.3523
Epoch [3/3], Loss: 1.3497
Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919
Metrics - Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919 for {'learning_rate': 0.001, 'batch_size': 32}
Testing parameters: {'learning_rate': 0.001, 'batch_size': 64}
Epoch [1/3], Loss: 1.5053
Epoch [2/3], Loss: 1.3577
Epoch [3/3], Loss: 1.3549
Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919
Metrics - Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919 for {'learning_rate': 0.001, 'batch_size': 64}
Testing parameters: {'learning_rate': 0.005, 'batch_size': 32}
Epoch [1/3], Loss: 68689.3991
Epoch [2/3], Loss: 113.1936
Epoch [3/3], Loss: 157.0089
Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919
Metrics - Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919 for {'learning_rate': 0.005, 'batch_size': 32}
Testing parameters: {'learning_rate': 0.005, 'batch_size': 64}
Epoch [1/3], Loss: 1362

In [ ]:
# Finding best optimizer on 0.001 learning rate, batch_size 16, and epochs 2, 3, runtime 1318 minutes
# Testing parameters: {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'adam', 'epochs': 2}
# Epoch [1/2], Loss: 1.3769
# Epoch [2/2], Loss: 1.3528
# Accuracy: 18.78%
# Metrics - Accuracy: 18.78% for {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'adam', 'epochs': 2}

# Testing parameters: {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'adam', 'epochs': 3}
# Epoch [1/3], Loss: 1.3756
# Epoch [2/3], Loss: 1.3512
# Epoch [3/3], Loss: 1.3497
# Accuracy: 29.19%
# Metrics - Accuracy: 29.19% for {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'adam', 'epochs': 3}

# Testing parameters: {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'adamw', 'epochs': 2}
# Epoch [1/2], Loss: 1.4084
# Epoch [2/2], Loss: 1.3523
# Accuracy: 25.38%
# Metrics - Accuracy: 25.38% for {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'adamw', 'epochs': 2}

# Testing parameters: {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'adamw', 'epochs': 3}
# Epoch [1/3], Loss: 1.3808
# Epoch [2/3], Loss: 1.3912
# Epoch [3/3], Loss: 1.3550
# Accuracy: 29.19%
# Metrics - Accuracy: 29.19% for {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'adamw', 'epochs': 3}

# Testing parameters: {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'sgd', 'epochs': 2}
# Epoch [1/2], Loss: 1.3741
# Epoch [2/2], Loss: 1.3575
# Accuracy: 25.38%
# Metrics - Accuracy: 25.38% for {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'sgd', 'epochs': 2}

# Testing parameters: {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'sgd', 'epochs': 3}
# Epoch [1/3], Loss: 1.3728
# Epoch [2/3], Loss: 1.3566
# Epoch [3/3], Loss: 1.3515
# Accuracy: 25.38%
# Metrics - Accuracy: 25.38% for {'learning_rate': 0.001, 'batch_size': 16, 'optimizer': 'sgd', 'epochs': 3}

In [ ]:
# Finding best learning rate and batch_size, with 3 epochs and adam as optimizer, runtime 315 minutes
# Testing parameters: {'learning_rate': 0.001, 'batch_size': 32}
# Epoch [1/3], Loss: 1.4048
# Epoch [2/3], Loss: 1.3523
# Epoch [3/3], Loss: 1.3497
# Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919
# Metrics - Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919 for {'learning_rate': 0.001, 'batch_size': 32}
# Testing parameters: {'learning_rate': 0.001, 'batch_size': 64}
# Epoch [1/3], Loss: 1.5053
# Epoch [2/3], Loss: 1.3577
# Epoch [3/3], Loss: 1.3549
# Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919
# Metrics - Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919 for {'learning_rate': 0.001, 'batch_size': 64}
# Testing parameters: {'learning_rate': 0.005, 'batch_size': 32}
# Epoch [1/3], Loss: 68689.3991
# Epoch [2/3], Loss: 113.1936
# Epoch [3/3], Loss: 157.0089
# Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919
# Metrics - Accuracy: 29.19%, Precision: 0.2919, Recall: 0.2919 for {'learning_rate': 0.005, 'batch_size': 32}
# Testing parameters: {'learning_rate': 0.005, 'batch_size': 64}
# Epoch [1/3], Loss: 136287.3804
# Epoch [2/3], Loss: 1.3617
# Epoch [3/3], Loss: 1.3643
# Accuracy: 25.38%, Precision: 0.2538, Recall: 0.2538
# Metrics - Accuracy: 25.38%, Precision: 0.2538, Recall: 0.2538 for {'learning_rate': 0.005, 'batch_size': 64}
# Testing parameters: {'learning_rate': 0.01, 'batch_size': 32}
# Epoch [1/3], Loss: 494018392.6025
# Epoch [2/3], Loss: 1.4507
# Epoch [3/3], Loss: 1.3637
# Accuracy: 25.38%, Precision: 0.2538, Recall: 0.2538
# Metrics - Accuracy: 25.38%, Precision: 0.2538, Recall: 0.2538 for {'learning_rate': 0.01, 'batch_size': 32}
# Testing parameters: {'learning_rate': 0.01, 'batch_size': 64}
# Epoch [1/3], Loss: 637203317.0052
# Epoch [2/3], Loss: 393364.8050
# Epoch [3/3], Loss: 100684.0010
# Accuracy: 26.65%, Precision: 0.2665, Recall: 0.2665
# Metrics - Accuracy: 26.65%, Precision: 0.2665, Recall: 0.2665 for {'learning_rate': 0.01, 'batch_size': 64}
# Best Hyperparameters: {'learning_rate': 0.001, 'batch_size': 32}, Best Accuracy: 29.19%